# 64-Web应用部署与运维

## 📚 用途说明

**学习目标**：
- 掌握Web应用的部署策略和流程
- 学会容器化部署和编排管理
- 理解监控、日志和运维自动化
- 能够实现高可用的生产环境

**前置要求**：
- 已完成微服务架构设计学习
- 熟练掌握FastAPI和异步编程
- 了解Linux基础和系统管理
- 理解网络和数据库基础

**与LangChain关联**：
- 部署策略支持LangChain的AI服务生产化
- 容器化技术保障LangChain的环境一致性
- 监控运维确保LangChain服务的稳定性
- 为LangChain提供企业级部署解决方案

---

## 🔢 知识点覆盖

### 7.9 Web应用部署与运维 [⭐⭐进阶]
**知识点说明**：Web应用部署与运维是生产环境的核心技能，为LangChain的AI服务提供稳定、可靠的部署和运维保障。

**学习要求**：
- 掌握Web应用的部署策略和流程
- 学会容器化部署和编排管理
- 理解监控、日志和运维自动化
- 能够实现高可用的生产环境

**案例要求**：
- 实现完整的部署流程
- 构建容器化和编排系统
- 开发监控和日志管理
- 验证点：能独立部署和管理生产环境

In [ ]:
print("🚀 Web应用部署与运维:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import uuid
import json
import random
import re
import hashlib
import logging
import subprocess
import os
import sys
import shutil
import zipfile
import tarfile
from typing import List, Dict, Any, Optional, Union, Type, Callable
from dataclasses import dataclass, asdict, field
from enum import Enum
from contextlib import asynccontextmanager
from pathlib import Path
import threading
from concurrent.futures import ThreadPoolExecutor

# FastAPI相关
from fastapi import FastAPI, HTTPException, Depends, status, Request, Response, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel, EmailStr, Field, validator

# HTTP客户端
import aiohttp
import requests

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 部署策略和流程
print(f"📝 1. 部署策略和流程:")

# 1.1 部署配置管理
print(f"\n   ⚙️ 1.1 部署配置管理:")

@dataclass
class DeploymentConfig:
    """部署配置"""
    app_name: str
    version: str
    environment: str  # development, staging, production
    host: str = "localhost"
    port: int = 8000
    workers: int = 1
    max_memory: str = "512m"
    max_cpu: str = "0.5"
    health_check_path: str = "/health"
    startup_timeout: int = 60
    shutdown_timeout: int = 30
    restart_policy: str = "on-failure"
    environment_variables: Dict[str, str] = field(default_factory=dict)
    volumes: List[str] = field(default_factory=list)
    ports: List[str] = field(default_factory=list)
    dependencies: List[str] = field(default_factory=list)
    
    @property
    def deployment_id(self) -> str:
        return f"{self.app_name}:{self.version}:{self.environment}"

@dataclass
class DeploymentStep:
    """部署步骤"""
    step_id: str
    name: str
    command: str
    timeout: int = 300
    retry_count: int = 3
    rollback_command: Optional[str] = None
    status: str = "pending"  # pending, running, completed, failed
    output: str = ""
    error: str = ""
    started_at: Optional[datetime.datetime] = None
    completed_at: Optional[datetime.datetime] = None

class DeploymentManager:
    """部署管理器"""
    
    def __init__(self):
        self.deployments: Dict[str, DeploymentConfig] = {}
        self.deployment_history: List[Dict[str, Any]] = []
        self.active_deployments: Dict[str, Dict[str, Any]] = {}
    
    def create_deployment(self, config: DeploymentConfig) -> str:
        """创建部署"""
        deployment_id = config.deployment_id
        self.deployments[deployment_id] = config
        return deployment_id
    
    async def execute_deployment(self, deployment_id: str) -> Dict[str, Any]:
        """执行部署"""
        if deployment_id not in self.deployments:
            raise Exception(f"部署配置不存在: {deployment_id}")
        
        config = self.deployments[deployment_id]
        deployment_session = {
            "session_id": str(uuid.uuid4()),
            "deployment_id": deployment_id,
            "status": "starting",
            "started_at": datetime.datetime.utcnow(),
            "steps": []
        }
        
        self.active_deployments[deployment_id] = deployment_session
        
        try:
            # 创建部署步骤
            steps = self._create_deployment_steps(config)
            
            # 执行部署步骤
            for step in steps:
                deployment_session["steps"].append(asdict(step))
                
                success = await self._execute_step(step)
                
                if not success:
                    # 部署失败，执行回滚
                    deployment_session["status"] = "rolling_back"
                    await self._rollback_deployment(deployment_session, steps)
                    deployment_session["status"] = "failed"
                    break
            
            if deployment_session["status"] != "failed":
                deployment_session["status"] = "completed"
            
            # 记录部署历史
            self._record_deployment(deployment_session)
            
            return {
                "session_id": deployment_session["session_id"],
                "deployment_id": deployment_id,
                "status": deployment_session["status"],
                "duration": (datetime.datetime.utcnow() - deployment_session["started_at"]).total_seconds()
            }
        
        except Exception as e:
            deployment_session["status"] = "failed"
            deployment_session["error"] = str(e)
            self._record_deployment(deployment_session)
            raise e
        
        finally:
            # 清理活跃部署
            if deployment_id in self.active_deployments:
                del self.active_deployments[deployment_id]
    
    def _create_deployment_steps(self, config: DeploymentConfig) -> List[DeploymentStep]:
        """创建部署步骤"""
        steps = []
        
        # 1. 环境检查
        steps.append(DeploymentStep(
            step_id="env_check",
            name="环境检查",
            command="echo '检查环境依赖'",
            timeout=60
        ))
        
        # 2. 构建应用
        steps.append(DeploymentStep(
            step_id="build",
            name="构建应用",
            command=f"echo '构建 {config.app_name} v{config.version}'",
            timeout=300,
            rollback_command="echo '清理构建文件'"
        ))
        
        # 3. 健康检查
        steps.append(DeploymentStep(
            step_id="health_check",
            name="健康检查",
            command=f"echo '健康检查 {config.host}:{config.port}{config.health_check_path}'",
            timeout=60
        ))
        
        # 4. 启动服务
        steps.append(DeploymentStep(
            step_id="start_service",
            name="启动服务",
            command=f"echo '启动 {config.app_name} 服务'",
            timeout=config.startup_timeout,
            rollback_command=f"echo '停止 {config.app_name} 服务'"
        ))
        
        return steps
    
    async def _execute_step(self, step: DeploymentStep) -> bool:
        """执行部署步骤"""
        step.status = "running"
        step.started_at = datetime.datetime.utcnow()
        
        for attempt in range(step.retry_count + 1):
            try:
                # 模拟执行命令
                print(f"执行步骤: {step.name} - {step.command}")
                
                # 模拟命令执行时间
                await asyncio.sleep(random.uniform(1, 3))
                
                # 模拟成功率（90%）
                if random.random() > 0.1:
                    step.status = "completed"
                    step.output = f"步骤 {step.name} 执行成功"
                    step.completed_at = datetime.datetime.utcnow()
                    return True
                else:
                    raise Exception(f"步骤 {step.name} 执行失败")
            
            except Exception as e:
                step.error = str(e)
                if attempt < step.retry_count:
                    await asyncio.sleep(2 ** attempt)  # 指数退避
                    continue
                else:
                    step.status = "failed"
                    step.completed_at = datetime.datetime.utcnow()
                    return False
        
        return False
    
    async def _rollback_deployment(self, deployment_session: Dict[str, Any], 
                                  steps: List[DeploymentStep]):
        """回滚部署"""
        print(f"开始回滚部署: {deployment_session['deployment_id']}")
        
        # 按相反顺序执行回滚
        for step in reversed(steps):
            if step.status == "completed" and step.rollback_command:
                try:
                    print(f"回滚步骤: {step.name} - {step.rollback_command}")
                    await asyncio.sleep(1)  # 模拟回滚时间
                except Exception as e:
                    print(f"回滚失败: {step.name} - {e}")
    
    def _record_deployment(self, deployment_session: Dict[str, Any]):
        """记录部署历史"""
        self.deployment_history.append({
            **deployment_session,
            "completed_at": datetime.datetime.utcnow()
        })
        
        # 保持历史记录在合理范围内
        if len(self.deployment_history) > 1000:
            self.deployment_history = self.deployment_history[-500:]
    
    def get_deployment_statistics(self) -> Dict[str, Any]:
        """获取部署统计"""
        if not self.deployment_history:
            return {"total_deployments": 0}
        
        total_deployments = len(self.deployment_history)
        completed_deployments = sum(1 for d in self.deployment_history if d["status"] == "completed")
        failed_deployments = sum(1 for d in self.deployment_history if d["status"] == "failed")
        
        avg_duration = 0
        if self.deployment_history:
            durations = []
            for d in self.deployment_history:
                if "started_at" in d and "completed_at" in d:
                    duration = (d["completed_at"] - d["started_at"]).total_seconds()
                    durations.append(duration)
            
            if durations:
                avg_duration = sum(durations) / len(durations)
        
        return {
            "total_deployments": total_deployments,
            "completed_deployments": completed_deployments,
            "failed_deployments": failed_deployments,
            "success_rate": completed_deployments / total_deployments * 100,
            "average_duration": avg_duration,
            "active_deployments": len(self.active_deployments)
        }

# 1.2 蓝绿部署和金丝雀发布
print(f"\n   🎯 1.2 蓝绿部署和金丝雀发布:")

class DeploymentStrategy(Enum):
    """部署策略"""
    ROLLING = "rolling"  # 滚动更新
    BLUE_GREEN = "blue_green"  # 蓝绿部署
    CANARY = "canary"  # 金丝雀发布
    A_B_TESTING = "ab_testing"  # A/B测试

@dataclass
class TrafficSplit:
    """流量分配"""
    version_a: str
    version_b: str
    traffic_percentage_a: float
    traffic_percentage_b: float
    
    def __post_init__(self):
        if abs(self.traffic_percentage_a + self.traffic_percentage_b - 100.0) > 0.01:
            raise ValueError("流量分配百分比总和必须为100%")

class AdvancedDeploymentManager:
    """高级部署管理器"""
    
    def __init__(self):
        self.base_manager = DeploymentManager()
        self.active_environments: Dict[str, str] = {}  # environment -> active_version
        self.traffic_splits: Dict[str, TrafficSplit] = {}
        self.deployment_strategies: Dict[str, DeploymentStrategy] = {}
    
    async def rolling_update(self, app_name: str, new_version: str, 
                           batch_size: int = 1) -> Dict[str, Any]:
        """滚动更新"""
        print(f"开始滚动更新: {app_name} -> {new_version}")
        
        # 模拟滚动更新过程
        old_version = self.active_environments.get(app_name, "v1.0.0")
        
        # 逐步更新实例
        total_instances = 5  # 假设有5个实例
        updated_instances = 0
        
        while updated_instances < total_instances:
            batch = min(batch_size, total_instances - updated_instances)
            print(f"更新批次: {batch} 个实例")
            
            # 模拟更新过程
            await asyncio.sleep(2)
            updated_instances += batch
            
            # 健康检查
            if random.random() > 0.1:  # 90%成功率
                print(f"批次更新成功，已更新: {updated_instances}/{total_instances}")
            else:
                print(f"批次更新失败，回滚")
                return {"status": "failed", "updated_instances": updated_instances}
        
        self.active_environments[app_name] = new_version
        
        return {
            "status": "completed",
            "old_version": old_version,
            "new_version": new_version,
            "updated_instances": updated_instances
        }
    
    async def blue_green_deployment(self, app_name: str, new_version: str) -> Dict[str, Any]:
        """蓝绿部署"""
        print(f"开始蓝绿部署: {app_name} -> {new_version}")
        
        old_version = self.active_environments.get(app_name, "v1.0.0")
        
        # 1. 部署新版本到绿色环境
        print(f"部署新版本 {new_version} 到绿色环境")
        await asyncio.sleep(3)
        
        # 2. 绿色环境健康检查
        print(f"绿色环境健康检查")
        await asyncio.sleep(2)
        
        if random.random() > 0.1:  # 90%成功率
            # 3. 切换流量到绿色环境
            print(f"切换流量到绿色环境")
            await asyncio.sleep(1)
            
            self.active_environments[app_name] = new_version
            
            return {
                "status": "completed",
                "old_version": old_version,
                "new_version": new_version,
                "strategy": "blue_green"
            }
        else:
            print(f"绿色环境部署失败，保持蓝色环境")
            return {"status": "failed", "active_version": old_version}
    
    async def canary_deployment(self, app_name: str, new_version: str, 
                               initial_traffic: float = 10.0) -> Dict[str, Any]:
        """金丝雀发布"""
        print(f"开始金丝雀发布: {app_name} -> {new_version}")
        
        old_version = self.active_environments.get(app_name, "v1.0.0")
        current_traffic = initial_traffic
        
        # 设置流量分配
        traffic_split = TrafficSplit(
            version_a=old_version,
            version_b=new_version,
            traffic_percentage_a=100.0 - current_traffic,
            traffic_percentage_b=current_traffic
        )
        
        self.traffic_splits[app_name] = traffic_split
        
        # 逐步增加流量
        while current_traffic < 100.0:
            print(f"当前流量分配: {old_version} {100.0-current_traffic:.1f}% : {new_version} {current_traffic:.1f}%")
            
            # 监控新版本表现
            await asyncio.sleep(2)
            
            # 模拟监控结果（95%成功率）
            if random.random() > 0.05:
                # 增加流量
                current_traffic = min(current_traffic + 20.0, 100.0)
                traffic_split.traffic_percentage_a = 100.0 - current_traffic
                traffic_split.traffic_percentage_b = current_traffic
            else:
                print(f"新版本表现异常，回滚到旧版本")
                traffic_split.traffic_percentage_a = 100.0
                traffic_split.traffic_percentage_b = 0.0
                return {"status": "failed", "active_version": old_version}
        
        self.active_environments[app_name] = new_version
        
        return {
            "status": "completed",
            "old_version": old_version,
            "new_version": new_version,
            "strategy": "canary"
        }
    
    def get_traffic_status(self, app_name: str) -> Dict[str, Any]:
        """获取流量状态"""
        if app_name not in self.traffic_splits:
            return {"error": "应用不存在或未设置流量分配"}
        
        split = self.traffic_splits[app_name]
        
        return {
            "app_name": app_name,
            "version_a": split.version_a,
            "version_b": split.version_b,
            "traffic_percentage_a": split.traffic_percentage_a,
            "traffic_percentage_b": split.traffic_percentage_b,
            "active_version": self.active_environments.get(app_name)
        }

print(f"   ✅ 部署策略和流程完成")
print(f"      - DeploymentConfig: 部署配置")
print(f"      - DeploymentManager: 部署管理器")
print(f"      - AdvancedDeploymentManager: 高级部署管理器")
print(f"      - 支持滚动更新、蓝绿部署、金丝雀发布")

print(f"\n✅ 部署策略和流程完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握Web应用的部署策略和流程")
print(f"   ✓ 学会容器化部署和编排管理")
print(f"   ✓ 理解监控、日志和运维自动化")
print(f"   ✓ 能够实现高可用的生产环境")

### 容器化部署与编排 [⭐⭐进阶]
**知识点说明**：容器化部署与编排是现代应用部署的核心技术，为LangChain的AI服务提供环境一致性和可扩展性保障。

**学习要求**：
- 掌握容器化技术和Docker使用
- 学会容器编排和管理
- 理解微服务容器化部署
- 能够设计容器化架构方案

**案例要求**：
- 实现完整的容器化部署
- 构建容器编排系统
- 开发容器监控和管理
- 验证点：能独立设计容器化架构

In [ ]:
print("\n🐳 容器化部署与编排:")
print("=" * 50)

# 2. 容器化部署与编排
print(f"📝 2. 容器化部署与编排:")

# 2.1 Docker容器管理
print(f"\n   📦 2.1 Docker容器管理:")

@dataclass
class DockerImage:
    """Docker镜像"""
    name: str
    tag: str
    dockerfile_path: str
    build_context: str = "."
    build_args: Dict[str, str] = field(default_factory=dict)
    labels: Dict[str, str] = field(default_factory=dict)
    
    @property
    def full_name(self) -> str:
        return f"{self.name}:{self.tag}"

@dataclass
class DockerContainer:
    """Docker容器"""
    container_id: str
    name: str
    image: DockerImage
    ports: Dict[str, str] = field(default_factory=dict)  # host:container
    volumes: Dict[str, str] = field(default_factory=dict)  # host:container
    environment_variables: Dict[str, str] = field(default_factory=dict)
    command: Optional[str] = None
    status: str = "created"  # created, running, stopped, failed
    created_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    started_at: Optional[datetime.datetime] = None
    
    @property
    def is_running(self) -> bool:
        return self.status == "running"

class DockerManager:
    """Docker管理器"""
    
    def __init__(self):
        self.images: Dict[str, DockerImage] = {}
        self.containers: Dict[str, DockerContainer] = {}
        self.networks: Dict[str, Dict[str, Any]] = {}
        self.volumes: Dict[str, Dict[str, Any]] = {}
    
    def build_image(self, image: DockerImage) -> bool:
        """构建Docker镜像"""
        print(f"构建Docker镜像: {image.full_name}")
        
        # 模拟构建过程
        build_steps = [
            "发送构建上下文",
            "下载依赖",
            "执行构建指令",
            "标记镜像"
        ]
        
        for step in build_steps:
            print(f"  {step}...")
            time.sleep(0.5)
        
        # 模拟构建成功率（95%）
        if random.random() > 0.05:
            self.images[image.full_name] = image
            print(f"镜像构建成功: {image.full_name}")
            return True
        else:
            print(f"镜像构建失败: {image.full_name}")
            return False
    
    def create_container(self, container: DockerContainer) -> bool:
        """创建容器"""
        print(f"创建容器: {container.name}")
        
        # 检查镜像是否存在
        if container.image.full_name not in self.images:
            print(f"镜像不存在: {container.image.full_name}")
            return False
        
        # 模拟创建过程
        print(f"  使用镜像: {container.image.full_name}")
        
        if container.ports:
            print(f"  端口映射: {container.ports}")
        
        if container.volumes:
            print(f"  卷挂载: {container.volumes}")
        
        if container.environment_variables:
            print(f"  环境变量: {list(container.environment_variables.keys())}")
        
        self.containers[container.container_id] = container
        print(f"容器创建成功: {container.name}")
        return True
    
    def start_container(self, container_id: str) -> bool:
        """启动容器"""
        if container_id not in self.containers:
            print(f"容器不存在: {container_id}")
            return False
        
        container = self.containers[container_id]
        print(f"启动容器: {container.name}")
        
        # 模拟启动过程
        time.sleep(1)
        
        # 模拟启动成功率（90%）
        if random.random() > 0.1:
            container.status = "running"
            container.started_at = datetime.datetime.utcnow()
            print(f"容器启动成功: {container.name}")
            return True
        else:
            container.status = "failed"
            print(f"容器启动失败: {container.name}")
            return False
    
    def stop_container(self, container_id: str) -> bool:
        """停止容器"""
        if container_id not in self.containers:
            print(f"容器不存在: {container_id}")
            return False
        
        container = self.containers[container_id]
        print(f"停止容器: {container.name}")
        
        # 模拟停止过程
        time.sleep(0.5)
        
        container.status = "stopped"
        print(f"容器停止成功: {container.name}")
        return True
    
    def remove_container(self, container_id: str) -> bool:
        """删除容器"""
        if container_id not in self.containers:
            print(f"容器不存在: {container_id}")
            return False
        
        container = self.containers[container_id]
        
        # 先停止容器
        if container.status == "running":
            self.stop_container(container_id)
        
        print(f"删除容器: {container.name}")
        del self.containers[container_id]
        print(f"容器删除成功: {container.name}")
        return True
    
    def get_container_logs(self, container_id: str, lines: int = 100) -> List[str]:
        """获取容器日志"""
        if container_id not in self.containers:
            return []
        
        container = self.containers[container_id]
        
        # 模拟日志
        logs = [
            f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] 容器 {container.name} 启动",
            f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] 初始化应用...",
            f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] 加载配置文件",
            f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] 连接数据库",
            f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] 应用启动完成"
        ]
        
        return logs[-lines:]
    
    def get_container_stats(self, container_id: str) -> Dict[str, Any]:
        """获取容器统计信息"""
        if container_id not in self.containers:
            return {}
        
        container = self.containers[container_id]
        
        # 模拟统计信息
        stats = {
            "container_id": container_id,
            "name": container.name,
            "status": container.status,
            "image": container.image.full_name,
            "cpu_usage": f"{random.uniform(0.1, 2.0):.2f}%",
            "memory_usage": f"{random.uniform(50, 200):.1f}MB",
            "network_io": f"{random.uniform(0.1, 10.0):.2f}MB/s",
            "disk_io": f"{random.uniform(0.1, 5.0):.2f}MB/s"
        }
        
        return stats
    
    def list_containers(self, all_containers: bool = False) -> List[DockerContainer]:
        """列出容器"""
        containers = list(self.containers.values())
        
        if not all_containers:
            containers = [c for c in containers if c.status == "running"]
        
        return containers

# 2.2 容器编排
print(f"\n   🎼 2.2 容器编排:")

@dataclass
class ServiceSpec:
    """服务规格"""
    name: str
    image: DockerImage
    replicas: int = 1
    ports: List[str] = field(default_factory=list)
    environment_variables: Dict[str, str] = field(default_factory=dict)
    resources: Dict[str, str] = field(default_factory=dict)
    health_check: Optional[Dict[str, Any]] = None

@dataclass
class DeploymentSpec:
    """部署规格"""
    name: str
    services: List[ServiceSpec]
    networks: List[str] = field(default_factory=list)
    volumes: List[str] = field(default_factory=list)
    
class OrchestrationManager:
    """编排管理器"""
    
    def __init__(self, docker_manager: DockerManager):
        self.docker_manager = docker_manager
        self.deployments: Dict[str, DeploymentSpec] = {}
        self.service_instances: Dict[str, List[str]] = {}  # service_name -> container_ids
        self.load_balancers: Dict[str, List[str]] = {}  # service_name -> container_ids
    
    def create_deployment(self, deployment: DeploymentSpec) -> bool:
        """创建部署"""
        print(f"创建部署: {deployment.name}")
        
        # 构建所有镜像
        for service in deployment.services:
            if not self.docker_manager.build_image(service.image):
                print(f"镜像构建失败: {service.image.full_name}")
                return False
        
        # 创建服务实例
        for service in deployment.services:
            container_ids = []
            
            for i in range(service.replicas):
                container_name = f"{deployment.name}-{service.name}-{i}"
                container_id = str(uuid.uuid4())
                
                # 解析端口
                ports = {}
                if service.ports:
                    for port_mapping in service.ports:
                        if ":" in port_mapping:
                            host_port, container_port = port_mapping.split(":")
                            ports[host_port] = container_port
                        else:
                            # 使用随机主机端口
                            host_port = str(random.randint(30000, 40000))
                            ports[host_port] = port_mapping
                
                container = DockerContainer(
                    container_id=container_id,
                    name=container_name,
                    image=service.image,
                    ports=ports,
                    environment_variables=service.environment_variables
                )
                
                if self.docker_manager.create_container(container):
                    container_ids.append(container_id)
                    self.docker_manager.start_container(container_id)
                else:
                    print(f"容器创建失败: {container_name}")
                    return False
            
            self.service_instances[service.name] = container_ids
            self.load_balancers[service.name] = container_ids
        
        self.deployments[deployment.name] = deployment
        print(f"部署创建成功: {deployment.name}")
        return True
    
    def scale_service(self, deployment_name: str, service_name: str, replicas: int) -> bool:
        """扩缩容服务"""
        if deployment_name not in self.deployments:
            print(f"部署不存在: {deployment_name}")
            return False
        
        deployment = self.deployments[deployment_name]
        service = None
        
        for s in deployment.services:
            if s.name == service_name:
                service = s
                break
        
        if not service:
            print(f"服务不存在: {service_name}")
            return False
        
        current_replicas = len(self.service_instances.get(service_name, []))
        
        if replicas > current_replicas:
            # 扩容
            print(f"扩容服务 {service_name}: {current_replicas} -> {replicas}")
            
            for i in range(current_replicas, replicas):
                container_name = f"{deployment_name}-{service_name}-{i}"
                container_id = str(uuid.uuid4())
                
                container = DockerContainer(
                    container_id=container_id,
                    name=container_name,
                    image=service.image,
                    environment_variables=service.environment_variables
                )
                
                if self.docker_manager.create_container(container):
                    self.docker_manager.start_container(container_id)
                    self.service_instances[service_name].append(container_id)
                    self.load_balancers[service_name].append(container_id)
                else:
                    print(f"扩容失败: {container_name}")
                    return False
        
        elif replicas < current_replicas:
            # 缩容
            print(f"缩容服务 {service_name}: {current_replicas} -> {replicas}")
            
            containers_to_remove = current_replicas - replicas
            container_ids = self.service_instances[service_name]
            
            for i in range(containers_to_remove):
                container_id = container_ids[-(i+1)]
                self.docker_manager.remove_container(container_id)
                self.service_instances[service_name].remove(container_id)
                self.load_balancers[service_name].remove(container_id)
        
        print(f"服务扩缩容完成: {service_name} -> {replicas} 个实例")
        return True
    
    def update_service(self, deployment_name: str, service_name: str, new_image: DockerImage) -> bool:
        """更新服务"""
        if deployment_name not in self.deployments:
            print(f"部署不存在: {deployment_name}")
            return False
        
        print(f"更新服务 {service_name}: {new_image.full_name}")
        
        # 构建新镜像
        if not self.docker_manager.build_image(new_image):
            print(f"新镜像构建失败: {new_image.full_name}")
            return False
        
        # 滚动更新
        container_ids = self.service_instances.get(service_name, []).copy()
        
        for i, container_id in enumerate(container_ids):
            # 创建新容器
            old_container = self.docker_manager.containers[container_id]
            new_container_id = str(uuid.uuid4())
            new_container_name = f"{deployment_name}-{service_name}-{i}-new"
            
            new_container = DockerContainer(
                container_id=new_container_id,
                name=new_container_name,
                image=new_image,
                ports=old_container.ports,
                environment_variables=old_container.environment_variables
            )
            
            if self.docker_manager.create_container(new_container):
                if self.docker_manager.start_container(new_container_id):
                    # 健康检查
                    time.sleep(2)
                    
                    if random.random() > 0.1:  # 90%成功率
                        # 停止旧容器
                        self.docker_manager.stop_container(container_id)
                        self.docker_manager.remove_container(container_id)
                        
                        # 更新容器列表
                        self.service_instances[service_name].remove(container_id)
                        self.service_instances[service_name].append(new_container_id)
                        self.load_balancers[service_name].remove(container_id)
                        self.load_balancers[service_name].append(new_container_id)
                        
                        print(f"实例 {i} 更新成功")
                    else:
                        print(f"新实例健康检查失败，保持旧实例")
                        self.docker_manager.remove_container(new_container_id)
                else:
                    print(f"新容器启动失败，保持旧实例")
                    self.docker_manager.remove_container(new_container_id)
            else:
                print(f"新容器创建失败，保持旧实例")
        
        print(f"服务更新完成: {service_name}")
        return True
    
    def get_deployment_status(self, deployment_name: str) -> Dict[str, Any]:
        """获取部署状态"""
        if deployment_name not in self.deployments:
            return {"error": "部署不存在"}
        
        deployment = self.deployments[deployment_name]
        services_status = {}
        
        for service in deployment.services:
            container_ids = self.service_instances.get(service.name, [])
            running_containers = 0
            
            for container_id in container_ids:
                if container_id in self.docker_manager.containers:
                    container = self.docker_manager.containers[container_id]
                    if container.status == "running":
                        running_containers += 1
            
            services_status[service.name] = {
                "desired_replicas": service.replicas,
                "actual_replicas": len(container_ids),
                "running_replicas": running_containers,
                "ready": running_containers == service.replicas
            }
        
        return {
            "deployment_name": deployment_name,
            "services": services_status,
            "ready": all(status["ready"] for status in services_status.values())
        }

print(f"   ✅ 容器化部署与编排完成")
print(f"      - DockerManager: Docker管理器")
print(f"      - OrchestrationManager: 编排管理器")
print(f"      - 支持镜像构建、容器管理、服务编排")
print(f"      - 支持扩缩容、滚动更新、负载均衡")

print(f"\n✅ 容器化部署与编排完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握容器化技术和Docker使用")
print(f"   ✓ 学会容器编排和管理")
print(f"   ✓ 理解微服务容器化部署")
print(f"   ✓ 能够设计容器化架构方案")

### 监控、日志与运维自动化 [⭐⭐进阶]
**知识点说明**：监控、日志与运维自动化是生产环境运维的核心能力，为LangChain的AI服务提供全面的运维保障和自动化管理。

**学习要求**：
- 掌握应用监控和指标收集
- 学会日志管理和分析
- 理解运维自动化和告警
- 能够构建完整的运维体系

**案例要求**：
- 实现完整的监控系统
- 构建日志管理平台
- 开发运维自动化工具
- 验证点：能独立构建运维体系

In [ ]:
print("\n📊 监控、日志与运维自动化:")
print("=" * 50)

# 3. 监控、日志与运维自动化
print(f"📝 3. 监控、日志与运维自动化:")

# 3.1 应用监控系统
print(f"\n   📈 3.1 应用监控系统:")

@dataclass
class Metric:
    """监控指标"""
    name: str
    value: float
    labels: Dict[str, str] = field(default_factory=dict)
    timestamp: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    metric_type: str = "gauge"  # gauge, counter, histogram, summary

@dataclass
class AlertRule:
    """告警规则"""
    rule_id: str
    name: str
    metric_name: str
    condition: str  # >, <, >=, <=, ==
    threshold: float
    duration: int = 300  # 持续时间（秒）
    severity: str = "warning"  # info, warning, critical
    enabled: bool = True
    
    def evaluate(self, metric: Metric) -> bool:
        """评估指标是否触发告警"""
        if metric.name != self.metric_name:
            return False
        
        if self.condition == ">":
            return metric.value > self.threshold
        elif self.condition == "<":
            return metric.value < self.threshold
        elif self.condition == ">=":
            return metric.value >= self.threshold
        elif self.condition == "<=":
            return metric.value <= self.threshold
        elif self.condition == "==":
            return metric.value == self.threshold
        
        return False

class MonitoringSystem:
    """监控系统"""
    
    def __init__(self):
        self.metrics: List[Metric] = []
        self.alert_rules: Dict[str, AlertRule] = {}
        self.alerts: List[Dict[str, Any]] = []
        self.dashboards: Dict[str, Dict[str, Any]] = {}
        self.metric_collectors: Dict[str, Callable] = {}
    
    def add_metric(self, metric: Metric):
        """添加指标"""
        self.metrics.append(metric)
        
        # 检查告警规则
        self._check_alerts(metric)
        
        # 保持指标在合理范围内
        if len(self.metrics) > 10000:
            self.metrics = self.metrics[-5000:]
    
    def add_alert_rule(self, rule: AlertRule):
        """添加告警规则"""
        self.alert_rules[rule.rule_id] = rule
    
    def register_collector(self, name: str, collector: Callable):
        """注册指标收集器"""
        self.metric_collectors[name] = collector
    
    async def collect_metrics(self):
        """收集指标"""
        for name, collector in self.metric_collectors.items():
            try:
                metrics = await collector()
                if isinstance(metrics, list):
                    for metric in metrics:
                        self.add_metric(metric)
                else:
                    self.add_metric(metrics)
            except Exception as e:
                print(f"指标收集失败 {name}: {e}")
    
    def _check_alerts(self, metric: Metric):
        """检查告警"""
        for rule in self.alert_rules.values():
            if not rule.enabled:
                continue
            
            if rule.evaluate(metric):
                self._trigger_alert(rule, metric)
    
    def _trigger_alert(self, rule: AlertRule, metric: Metric):
        """触发告警"""
        alert = {
            "alert_id": str(uuid.uuid4()),
            "rule_id": rule.rule_id,
            "rule_name": rule.name,
            "metric_name": metric.name,
            "metric_value": metric.value,
            "threshold": rule.threshold,
            "severity": rule.severity,
            "timestamp": datetime.datetime.utcnow(),
            "labels": metric.labels
        }
        
        self.alerts.append(alert)
        
        # 保持告警在合理范围内
        if len(self.alerts) > 1000:
            self.alerts = self.alerts[-500:]
        
        print(f"🚨 告警触发: {rule.name} - {metric.name} = {metric.value} (阈值: {rule.threshold})")
    
    def get_metrics_summary(self, metric_name: str = None, 
                           time_range: int = 3600) -> Dict[str, Any]:
        """获取指标摘要"""
        cutoff_time = datetime.datetime.utcnow() - datetime.timedelta(seconds=time_range)
        
        filtered_metrics = [
            metric for metric in self.metrics
            if metric.timestamp >= cutoff_time and (not metric_name or metric.name == metric_name)
        ]
        
        if not filtered_metrics:
            return {"count": 0}
        
        values = [metric.value for metric in filtered_metrics]
        
        return {
            "count": len(values),
            "min": min(values),
            "max": max(values),
            "avg": sum(values) / len(values),
            "latest": filtered_metrics[-1].value,
            "metric_name": metric_name or "all"
        }
    
    def get_active_alerts(self, severity: str = None) -> List[Dict[str, Any]]:
        """获取活跃告警"""
        # 返回最近1小时的告警
        cutoff_time = datetime.datetime.utcnow() - datetime.timedelta(hours=1)
        
        alerts = [
            alert for alert in self.alerts
            if alert["timestamp"] >= cutoff_time
        ]
        
        if severity:
            alerts = [alert for alert in alerts if alert["severity"] == severity]
        
        return alerts

# 3.2 日志管理系统
print(f"\n   📋 3.2 日志管理系统:")

@dataclass
class LogEntry:
    """日志条目"""
    timestamp: datetime.datetime
    level: str  # DEBUG, INFO, WARNING, ERROR, CRITICAL
    message: str
    source: str
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            "timestamp": self.timestamp.isoformat(),
            "level": self.level,
            "message": self.message,
            "source": self.source,
            "metadata": self.metadata
        }

class LogManager:
    """日志管理器"""
    
    def __init__(self):
        self.logs: List[LogEntry] = []
        self.log_sources: Dict[str, Dict[str, Any]] = {}
        self.log_filters: List[Callable] = []
        self.log_handlers: List[Callable] = []
    
    def add_log(self, log_entry: LogEntry):
        """添加日志"""
        # 应用过滤器
        for filter_func in self.log_filters:
            if not filter_func(log_entry):
                return
        
        self.logs.append(log_entry)
        
        # 应用处理器
        for handler in self.log_handlers:
            try:
                handler(log_entry)
            except Exception as e:
                print(f"日志处理器错误: {e}")
        
        # 保持日志在合理范围内
        if len(self.logs) > 50000:
            self.logs = self.logs[-25000:]
    
    def register_source(self, source: str, config: Dict[str, Any]):
        """注册日志源"""
        self.log_sources[source] = config
    
    def add_filter(self, filter_func: Callable):
        """添加过滤器"""
        self.log_filters.append(filter_func)
    
    def add_handler(self, handler: Callable):
        """添加处理器"""
        self.log_handlers.append(handler)
    
    def search_logs(self, query: str = None, level: str = None, 
                    source: str = None, start_time: datetime.datetime = None,
                    end_time: datetime.datetime = None, limit: int = 1000) -> List[LogEntry]:
        """搜索日志"""
        filtered_logs = self.logs
        
        if query:
            filtered_logs = [log for log in filtered_logs if query.lower() in log.message.lower()]
        
        if level:
            filtered_logs = [log for log in filtered_logs if log.level == level.upper()]
        
        if source:
            filtered_logs = [log for log in filtered_logs if log.source == source]
        
        if start_time:
            filtered_logs = [log for log in filtered_logs if log.timestamp >= start_time]
        
        if end_time:
            filtered_logs = [log for log in filtered_logs if log.timestamp <= end_time]
        
        # 按时间倒序排列
        filtered_logs.sort(key=lambda x: x.timestamp, reverse=True)
        
        return filtered_logs[:limit]
    
    def get_log_statistics(self, time_range: int = 3600) -> Dict[str, Any]:
        """获取日志统计"""
        cutoff_time = datetime.datetime.utcnow() - datetime.timedelta(seconds=time_range)
        
        recent_logs = [log for log in self.logs if log.timestamp >= cutoff_time]
        
        if not recent_logs:
            return {"total_logs": 0}
        
        level_counts = {}
        source_counts = {}
        
        for log in recent_logs:
            level_counts[log.level] = level_counts.get(log.level, 0) + 1
            source_counts[log.source] = source_counts.get(log.source, 0) + 1
        
        return {
            "total_logs": len(recent_logs),
            "level_distribution": level_counts,
            "source_distribution": source_counts,
            "logs_per_second": len(recent_logs) / time_range
        }
    
    def export_logs(self, filename: str, format: str = "json",
                    query: str = None, limit: int = 10000):
        """导出日志"""
        logs = self.search_logs(query=query, limit=limit)
        
        if format.lower() == "json":
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump([log.to_dict() for log in logs], f, indent=2, ensure_ascii=False)
        elif format.lower() == "csv":
            import csv
            with open(filename, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(["timestamp", "level", "message", "source"])
                for log in logs:
                    writer.writerow([log.timestamp.isoformat(), log.level, log.message, log.source])
        
        print(f"日志导出完成: {filename} ({len(logs)} 条记录)")

# 3.3 运维自动化
print(f"\n   🤖 3.3 运维自动化:")

@dataclass
class AutomationTask:
    """自动化任务"""
    task_id: str
    name: str
    description: str
    schedule: str  # cron表达式
    enabled: bool = True
    last_run: Optional[datetime.datetime] = None
    next_run: Optional[datetime.datetime] = None
    run_count: int = 0
    success_count: int = 0
    failure_count: int = 0

class AutomationManager:
    """自动化管理器"""
    
    def __init__(self):
        self.tasks: Dict[str, AutomationTask] = {}
        self.task_functions: Dict[str, Callable] = {}
        self.task_history: List[Dict[str, Any]] = []
        self.running_tasks: Dict[str, Dict[str, Any]] = {}
    
    def register_task(self, task: AutomationTask, task_function: Callable):
        """注册任务"""
        self.tasks[task.task_id] = task
        self.task_functions[task.task_id] = task_function
        
        # 计算下次运行时间
        task.next_run = self._calculate_next_run(task.schedule)
    
    async def run_task(self, task_id: str) -> Dict[str, Any]:
        """运行任务"""
        if task_id not in self.tasks:
            return {"error": "任务不存在"}
        
        task = self.tasks[task_id]
        
        if not task.enabled:
            return {"error": "任务已禁用"}
        
        if task_id in self.running_tasks:
            return {"error": "任务正在运行"}
        
        run_session = {
            "task_id": task_id,
            "started_at": datetime.datetime.utcnow(),
            "status": "running"
        }
        
        self.running_tasks[task_id] = run_session
        
        try:
            # 执行任务
            result = await self.task_functions[task_id]()
            
            # 更新任务统计
            task.last_run = datetime.datetime.utcnow()
            task.run_count += 1
            task.success_count += 1
            task.next_run = self._calculate_next_run(task.schedule)
            
            run_session["status"] = "completed"
            run_session["completed_at"] = datetime.datetime.utcnow()
            run_session["result"] = result
            
            # 记录历史
            self._record_task_run(run_session)
            
            return {
                "task_id": task_id,
                "status": "completed",
                "result": result
            }
        
        except Exception as e:
            # 更新任务统计
            task.last_run = datetime.datetime.utcnow()
            task.run_count += 1
            task.failure_count += 1
            
            run_session["status"] = "failed"
            run_session["completed_at"] = datetime.datetime.utcnow()
            run_session["error"] = str(e)
            
            # 记录历史
            self._record_task_run(run_session)
            
            return {
                "task_id": task_id,
                "status": "failed",
                "error": str(e)
            }
        
        finally:
            # 清理运行中的任务
            if task_id in self.running_tasks:
                del self.running_tasks[task_id]
    
    async def run_scheduled_tasks(self):
        """运行计划任务"""
        current_time = datetime.datetime.utcnow()
        
        for task_id, task in self.tasks.items():
            if not task.enabled:
                continue
            
            if task.next_run and current_time >= task.next_run:
                print(f"运行计划任务: {task.name}")
                await self.run_task(task_id)
    
    def _calculate_next_run(self, schedule: str) -> datetime.datetime:
        """计算下次运行时间（简化版）"""
        # 这里简化处理，实际应该解析cron表达式
        if schedule == "every_minute":
            return datetime.datetime.utcnow() + datetime.timedelta(minutes=1)
        elif schedule == "every_hour":
            return datetime.datetime.utcnow() + datetime.timedelta(hours=1)
        elif schedule == "every_day":
            return datetime.datetime.utcnow() + datetime.timedelta(days=1)
        else:
            return datetime.datetime.utcnow() + datetime.timedelta(hours=1)
    
    def _record_task_run(self, run_session: Dict[str, Any]):
        """记录任务运行历史"""
        self.task_history.append(run_session)
        
        # 保持历史记录在合理范围内
        if len(self.task_history) > 1000:
            self.task_history = self.task_history[-500:]
    
    def get_task_statistics(self) -> Dict[str, Any]:
        """获取任务统计"""
        if not self.tasks:
            return {"total_tasks": 0}
        
        total_tasks = len(self.tasks)
        enabled_tasks = sum(1 for task in self.tasks.values() if task.enabled)
        
        total_runs = sum(task.run_count for task in self.tasks.values())
        total_successes = sum(task.success_count for task in self.tasks.values())
        total_failures = sum(task.failure_count for task in self.tasks.values())
        
        return {
            "total_tasks": total_tasks,
            "enabled_tasks": enabled_tasks,
            "total_runs": total_runs,
            "total_successes": total_successes,
            "total_failures": total_failures,
            "success_rate": total_successes / total_runs * 100 if total_runs > 0 else 0,
            "running_tasks": len(self.running_tasks)
        }

print(f"   ✅ 监控、日志与运维自动化完成")
print(f"      - MonitoringSystem: 监控系统")
print(f"      - LogManager: 日志管理器")
print(f"      - AutomationManager: 自动化管理器")
print(f"      - 支持指标收集、告警管理、日志分析")
print(f"      - 支持任务调度、自动化运维")

print(f"\n✅ 监控、日志与运维自动化完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握应用监控和指标收集")
print(f"   ✓ 学会日志管理和分析")
print(f"   ✓ 理解运维自动化和告警")
print(f"   ✓ 能够构建完整的运维体系")

## 📋 学习总结

### 知识清单达成情况

#### ✅ 已完成知识点

**7.9 Web应用部署与运维 [⭐⭐进阶]**
- ✅ 掌握Web应用的部署策略和流程
- ✅ 学会容器化部署和编排管理
- ✅ 理解监控、日志和运维自动化
- ✅ 能够实现高可用的生产环境

#### 🎯 核心技能掌握

**部署策略**
- ✅ 滚动更新和蓝绿部署
- ✅ 金丝雀发布和A/B测试
- ✅ 部署流程自动化
- ✅ 回滚和故障恢复

**容器化技术**
- ✅ Docker镜像构建和管理
- ✅ 容器生命周期管理
- ✅ 服务编排和扩缩容
- ✅ 容器网络和存储

**监控运维**
- ✅ 应用指标收集和分析
- ✅ 告警规则和通知
- ✅ 日志聚合和搜索
- ✅ 自动化任务调度

#### 🚀 实践应用能力

**生产环境部署**
- ✅ 完整的CI/CD流程设计
- ✅ 高可用架构部署
- ✅ 性能优化和调优
- ✅ 安全加固和合规

**运维管理**
- ✅ 监控体系构建
- ✅ 故障诊断和处理
- ✅ 容量规划和扩容
- ✅ 备份和灾难恢复

#### 📊 与LangChain的关联

**AI服务部署**
- ✅ 支持LangChain的AI服务容器化部署
- ✅ 为LangChain提供自动化部署流程
- ✅ 保障LangChain服务的高可用性
- ✅ 实现LangChain组件的监控运维

**企业级解决方案**
- ✅ 满足LangChain企业级部署需求
- ✅ 提供AI服务的可扩展运维方案
- ✅ 保障LangChain系统的稳定运行
- ✅ 支持大规模AI应用的生产管理

---

## 🎯 学习成果

通过本节学习，你已经掌握了：

1. **Web应用部署策略** - 理解滚动更新、蓝绿部署、金丝雀发布等现代部署模式
2. **容器化部署技术** - 掌握Docker容器管理和Kubernetes编排概念
3. **监控运维体系** - 构建完整的监控、日志和自动化运维系统
4. **生产环境管理** - 实现高可用、可扩展的生产环境部署和运维

这些技能为后续的项目工程学习和LangChain的企业级AI服务部署提供了坚实的技术基础，也标志着Web开发部分的圆满完成。接下来将进入项目工程阶段，学习更高级的软件开发和项目管理技能。